## Project 1 (Beginner) — Content Site / Blog Platform  
### Workbook Guidelines (What to build, why, and how to verify) — without full spoonfed code

This section is written as a **project brief + execution guide** for learners. It
tells you *exactly what to build* and *how to know you built it correctly*, while
leaving implementation details for practice.

---

# 1) Project Goal (What you’re building)

Build a production-style **content site** with:

- Public pages: article list + article detail
- Editorial workflow: draft/publish, cover image, tags
- Comments: public submission + staff moderation
- Admin: usable content management
- SEO/ops basics: sitemap, robots.txt, canonical URLs
- Quality baseline: tests, security hygiene, performance hygiene (pagination + no N+1)

Think of it as: “a blog that could ship.”

---

# 2) Target Skills (What this project is meant to teach)

You should practice these Django skills end-to-end:

- URL design + view structure (FBV/CBV is your choice)
- Templates with inheritance + partials + safe rendering
- Models + migrations + constraints + relationships (M2M tags, FK comments)
- ORM filtering/search + pagination + query optimization
- Forms/ModelForms + validation + PRG pattern + messages UX
- Auth + authorization rules (staff vs author vs anonymous)
- Admin customization for real editorial workflows
- Testing for behavior + security + regression safety

---

# 3) Requirements (User Stories + Acceptance Criteria)

## 3.1 Public user stories
1. **Browse articles**
   - Visitor can open `/articles/` and see **published** articles only.
   - Articles are ordered newest-first (define what “newest” means: published_at or created_at).

2. **Search**
   - Visitor can search with `?q=...`.
   - Search does not crash on empty or long strings.
   - Search results are paginated.

3. **Filter by tag**
   - Visitor can filter by tag via query (`?tag=django`) OR tag pages (`/tags/django/`) (choose one for baseline; tag pages are a stretch goal).

4. **Open an article**
   - `/articles/<slug>/` renders:
     - title, author, published date
     - cover image if present
     - tags
     - body
   - Drafts are not visible publicly.

## 3.2 Editorial workflow stories
5. **Create article**
   - Authenticated user can create an article.
   - Author is set by the server (not user input).
   - Default status is DRAFT (recommended baseline).

6. **Edit article**
   - Author can edit their own articles.
   - Staff can edit any article.
   - Non-author non-staff gets 403 or 404 (pick a policy and be consistent).

7. **Publish rules**
   - Define publish policy:
     - Option A (simpler): author can publish own article
     - Option B (more editorial): only staff can publish
   - Enforce the same rule in HTML and API (if you built API).

## 3.3 Comments stories
8. **Submit comment**
   - Anyone can submit a comment on a published article.
   - Comment requires name + body (min length).
   - On success: PRG redirect and “submitted for review” message.

9. **Moderate comments**
   - Comments are not visible until approved.
   - Staff can approve in Django admin (baseline).
   - Approved comments appear on article detail.

## 3.4 SEO/ops baseline
10. **Sitemap**
   - `/sitemap.xml` exists and lists published article URLs.

11. **robots.txt**
   - `/robots.txt` exists and disallows `/admin/` and `/accounts/`.

12. **Canonical URL**
   - Article detail includes canonical link tag.

## 3.5 Quality baseline
13. **Tests exist**
   - At minimum: visibility rules (draft vs published), list pagination, comment submission, permissions.

14. **Performance hygiene**
   - Article list is paginated.
   - No obvious N+1 when rendering tags/author.

15. **Security hygiene**
   - All POST forms include CSRF token.
   - Upload validation exists (size/type) if cover images enabled.

---

# 4) Data Model Guidelines (What models you likely need)

Minimum recommended models:

## 4.1 Article
Fields (typical):
- `title` (CharField)
- `slug` (SlugField, unique)
- `body` (TextField)
- `status` (choices: draft/published/archived)
- `published_at` (nullable)
- `author` (FK to user)
- `cover_image` (optional ImageField)
- timestamps: `created_at`, `updated_at`
- tags: ManyToMany to Tag
- constraints:
  - if status is published ⇒ published_at is not null (DB constraint recommended)

## 4.2 Tag
Fields:
- `name`
- `slug` unique

## 4.3 Comment
Fields:
- `article` FK
- `name`
- `body`
- `is_approved` boolean
- timestamps

**Guideline:** Keep comments simple and moderated by default. You can add user
accounts for commenters later, but baseline is anonymous comment submission.

---

# 5) URL Design Guidelines (Stable, predictable, “industry standard”)

Baseline URL map (recommended):

- `GET /articles/` list
- `GET /articles/<slug>/` detail
- `GET/POST /articles/new/` create (auth required)
- `GET/POST /articles/<slug>/edit/` edit (auth + authorization)
- `POST /articles/<slug>/comments/` comment submit
- `GET /tags/` (optional)
- `GET /tags/<slug>/` (optional)
- `GET /sitemap.xml`
- `GET /robots.txt`
- `GET /healthz`, `GET /readyz` (ops)

**Rule:** Put “special routes” (`new/`, `edit/`) before your catch-all slug route.

---

# 6) Implementation Plan (Milestones)

Each milestone has:
- deliverables
- recommended approach
- “Definition of Done” checks

## Milestone 1 — Baseline setup and skeleton pages
Deliverables:
- Articles app exists
- `/articles/` returns HTML
- `/articles/<slug>/` returns HTML (even if hardcoded for now)

Definition of Done:
- routes are namespaced and reversed via `{% url %}`
- base template and nav exist

---

## Milestone 2 — Models + migrations + admin
Deliverables:
- Article, Tag models created and migrated
- admin registration with:
  - list_display, filters, search
  - prepopulated slug
  - tag selection UI usable

Definition of Done:
- you can create tags + articles via admin
- you can filter/search articles in admin without pain

Guidelines:
- add indexes for slug and common list ordering
- add a constraint for publish timestamp (recommended)

---

## Milestone 3 — Public article list (filter/search/pagination)
Deliverables:
- `/articles/` displays published-only
- supports `?q=...` and `?tag=...` (or tag pages)
- paginated

Definition of Done:
- invalid `page` inputs don’t crash (fallback to page 1 or last page)
- query count does not explode when rendering tags

Hints (not full code):
- use `Paginator(queryset, page_size)`
- build queryset progressively:
  - start with `Article.objects.published()`
  - apply tag filter if tag present
  - apply OR search with `Q(title__icontains=q) | Q(body__icontains=q)`
  - `.distinct()` after M2M joins
- optimize:
  - `select_related("author")`
  - `prefetch_related("tags")`

---

## Milestone 4 — Article detail + draft visibility rules
Deliverables:
- `/articles/<slug>/` shows article
- public can only see published
- author/staff can preview drafts (policy choice)

Definition of Done:
- anonymous user hitting a draft returns 404 or 403 (document your choice)
- template safely escapes content (no `|safe` on user content)

Hints:
- if you want “draft hidden,” use a queryset that already enforces visibility
- keep the visibility rule centralized (selector or policy function)

---

## Milestone 5 — Create/edit forms + messages + PRG
Deliverables:
- `/articles/new/` create
- `/articles/<slug>/edit/` edit
- uses ModelForm validation
- success uses redirect + message

Definition of Done:
- double-submit doesn’t create duplicates easily (PRG prevents refresh resubmits)
- author is set server-side (`commit=False`)
- edit page enforces authorization server-side

Hints:
- Create: `form.save(commit=False)` then set `author`, then `save()`, then `save_m2m()`
- Edit: `instance=article` form pattern
- use `messages.success(...)`

---

## Milestone 6 — Comments with moderation
Deliverables:
- comment form on article detail
- POST endpoint creates comment with `is_approved=False`
- admin can approve comments
- article detail shows approved comments only

Definition of Done:
- invalid comment re-renders detail with errors (recommended)
- comment spam is reduced (at least honeypot or rate limit concept)

Hints:
- use PRG for success
- for invalid comment, re-render detail template with bound form + errors

---

## Milestone 7 — SEO/ops basics
Deliverables:
- sitemap and robots.txt
- canonical URLs
- (optional) OpenGraph/Twitter meta tags

Definition of Done:
- sitemap loads quickly and lists only published articles
- robots.txt disallows admin/accounts
- canonical link matches the actual article URL

Hints:
- implement `get_absolute_url()` on Article
- wrap sitemap view with caching (short TTL is okay)

---

# 7) Testing Guidelines (What to test, minimum set)

Minimum test suite (high value):

## 7.1 Visibility policy tests
- published visible to anonymous
- draft hidden to anonymous
- draft visible to author/staff (if you support preview)

## 7.2 List behavior tests
- `/articles/` returns 200
- search returns expected article(s)
- pagination works, bad `page` handled

## 7.3 Comment workflow tests
- posting a valid comment creates comment (unapproved)
- approved comment appears on page
- invalid comment shows errors

## 7.4 Authorization tests
- create requires login
- non-owner cannot edit
- staff can edit

## 7.5 Regression tests for query counts (optional but recommended)
- use `assertNumQueries` or pytest `django_assert_num_queries` on article list page
- goal: query count stays small and doesn’t grow with number of articles displayed

---

# 8) Security Guidelines (Must-do rules for Project 1)

- All POST forms include `{% csrf_token %}`
- Don’t render untrusted text with `|safe`
- Validate file uploads if you allow cover images:
  - size cap (e.g., 2MB)
  - content type allowlist (jpeg/png/webp) as a first layer
- Enforce permissions in backend:
  - never rely on “hide edit button” only
- Rate limit or at least add honeypot for comments (baseline anti-spam)

---

# 9) Performance Guidelines (What “good enough” means here)

- Public list endpoints must paginate (never return unbounded lists)
- Use:
  - `select_related("author")` on list/detail
  - `prefetch_related("tags")` on list/detail
- Use `.distinct()` when filtering by M2M tag join
- Cache safe public pages:
  - sitemap (e.g., 1 hour)
  - tag index (e.g., 5–15 minutes)

---

# 10) “Definition of Done” (Project Completion Gate)

Your Project 1 is complete when:

- [ ] Public:
  - [ ] `/articles/` lists published-only, paginated, supports search + tag filter
  - [ ] `/articles/<slug>/` works and is safe (no XSS risk patterns)
- [ ] Editorial:
  - [ ] Create/edit exists with validation + PRG + messages
  - [ ] Authorization rules implemented + tested
- [ ] Comments:
  - [ ] Submit works, moderation required, approved-only display
- [ ] Admin:
  - [ ] Tag and Article management is productive (filters/search/slug prepopulate)
- [ ] SEO/ops:
  - [ ] sitemap + robots + canonical exist
- [ ] Quality:
  - [ ] tests pass in CI-style run
  - [ ] ruff + black checks pass
  - [ ] no obvious N+1 on list/detail

---

# 11) Stretch Goals (Choose 2–4)

Pick a few to deepen learning:

1. Tag pages: `/tags/` and `/tags/<slug>/`
2. RSS feed: `/feeds/latest/`
3. Better pagination UI with page number window and querystring helper tag
4. Scheduled publishing (`publish_at`) + management command or Celery beat to publish
5. OpenGraph/Twitter meta + JSON-LD structured data
6. Comment spam controls:
   - honeypot
   - throttling
7. Search upgrade (PostgreSQL full-text search)

---

# 12) Common Failure Modes (and what to check)

- “Why does `/articles/new/` open article detail?”  
  Route ordering: `new/` must be before `<slug>/`.

- “Why do published pages show duplicates when filtering by tags?”  
  M2M join duplicates: add `.distinct()`.

- “Why does tag rendering cause the page to slow down?”  
  N+1: add `prefetch_related("tags")`.

- “Why is my form POST rejected?”  
  Missing CSRF token, missing multipart enctype for uploads, or missing `request.FILES`.

- “Why do drafts show publicly?”  
  Your queryset/policy is not enforcing status or visibility rules consistently.

---


<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../10. Advanced_topics/47. managing_large_codebases.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='49. saas_crud_platform.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
